# Scrapper pour obtenir les données sur les URLS facebook et Youtube

### importation des modules custom

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..')) +'\\analyser_tools'
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

C:\Users\tousi\Google Drive\radio_canada\projet\decrypteur\repo_pyMboxAnalyser\analyser_tools


In [10]:
from EmailDataFrame import *
from FacebookScrapper import FacebookScrapper
from YoutubeScrapper import YoutubeScrapper

### import python modules


In [12]:
import pathlib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import logging
from tqdm import tqdm
%matplotlib inline

In [6]:
### definir les paths vers les données

In [8]:
cwd = pathlib.PurePath(os.getcwd())
project_root_dir = cwd.parent
theme_path = project_root_dir / "data" / "theme_words.csv"
email_raw_path = project_root_dir / "output" / 'csv_file' / 'email_raw.csv'
facebook_url_info_path = project_root_dir / 'output' / 'csv_file' / 'facebook_urls_info.csv'
youtube_url_info_path = project_root_dir / 'output' / 'csv_file' / 'youtube_urls_info.csv'
log_file_path = project_root_dir / 'output' / 'log_file' / 'url_cleaning.log'
removed_email_path = project_root_dir / 'output' / 'csv_file' / 'removed_email_from_decrypteur.csv'
dataclean_pickle_path = project_root_dir / 'output' / 'pickle_obj' / 'dataClean'


### Load data


In [9]:
email_raw_converters = {"attach_type": lambda x: x.strip("[]").replace("'", "").lower().strip().split(", "),
                            "urls": lambda x: x.strip("[]").replace("'", "").strip().split(', ')
                            }
email_df_raw = EmailDF.from_csv(csv_path=email_raw_path, converters=email_raw_converters)
email_df_raw.read_dict_theme_from_csv(theme_path)

debut = (2020, 3, 1) # 1er mars 2020 pour le premier courriel
fin = (9999, 12, 30) # infini

email_df = (email_df_raw.filt_by_date(start_date=debut, end_date=fin) #conserver entre debut et fin
            .remove_email_by_from(logpath=removed_email_path) #log les courriels retirer 
            .clean_urls() #courriel spam (antivirus), repetition de domaine
            .add_domain_column() #trouver domaine des urls restants
            .add_has_url_column() #ajouter s'il y a des urls ou non
            .add_text_column() # combiner body et titre du courriel, puis retirer emoji, non-roman text, saut de ligne et tab
            .add_theme_column() # tagger les courriels à partir du text
            .add_source_column()) # trouver les sources (comme domaine, mais sans répétition)

 53%|███████████████████████████████████████▍                                   | 2753/5230 [00:00<00:00, 27330.18it/s]

creating domain column


  2%|█▌                                                                           | 103/5230 [00:00<00:05, 1022.52it/s]

creating has_url column


  1%|▍                                                                              | 33/5230 [00:00<00:16, 324.39it/s]

creating text column


  0%|▏                                                                              | 12/5230 [00:00<00:48, 108.37it/s]

tagging theme to email


100%|█████████████████████████████████████████████████████████████████████████████| 5230/5230 [00:27<00:00, 187.98it/s]

tagging source to email


### Scrapper 

In [13]:
print('parsing facebook urls')
facebook_urls = email_df.df.explode('urls')[email_df.df.domain.explode() == 'facebook']['urls'].value_counts()
data = []
for url in tqdm(facebook_urls.index):
    fs = FacebookScrapper(url, email_df.theme)
    fs.parse_data()
    data.append(fs.export_as_list())
df_urls_facebook = pd.DataFrame(data, columns = ['url', 'title', 'published_date', 'comments', 'reactions', 'shares', 'views', 'theme'])
df_urls_facebook.to_csv(facebook_url_info_path)


print('parsing youtube urls')
youtube_urls = email_df.df.explode('urls')[email_df.df.domain.explode() == 'youtube']['urls'].value_counts()
data = []
for url in tqdm(youtube_urls.index):
    fs = YoutubeScrapper(url, email_df.theme)
    fs.parse_data()
    data.append(fs.export_to_list())
df_urls_youtube = pd.DataFrame(data, columns = ['url', 'title', 'date_published', 'description', 'author', 'rating', 'views', 'theme'])
df_urls_youtube.to_csv(youtube_url_info_path)


  0%|                                                                                          | 0/872 [00:00<?, ?it/s]

parsing facebook urls


  0%|                                                                                          | 0/542 [00:00<?, ?it/s]

parsing youtube urls


100%|████████████████████████████████████████████████████████████████████████████████| 542/542 [44:18<00:00,  4.90s/it]
